## cnet

In [1]:
from sdserve.models.cnet.wrapper import ControlNetWrapper

/home/tiennv/.conda/envs/stable_py10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch


In [3]:
cnet = ControlNetWrapper(
    model_name_or_path="diffusers/controlnet-canny-sdxl-1.0", attention_slicing=None,
)

device = 'cpu'
dtype = torch.float32

In [4]:
dump_inputs = (
    torch.randn(2, 4, 128, 128).to(device=device, dtype=dtype),
    torch.randn(1).to(device=device, dtype=dtype),
    torch.randn(2, 77, 2048).to(device=device, dtype=dtype),
    torch.randn(2, 3, 1024, 1024).to(device=device, dtype=dtype),
    torch.randn(1).to(device=device, dtype=dtype), # controlnet_cond
    torch.randn(2, 1280, dtype=dtype, device=device), # add_text_embeds
    torch.randn(2, 6, dtype=dtype, device=device), # add_time_ids
)

module = torch.jit.trace(cnet, dump_inputs)

/home/tiennv/.conda/envs/stable_py10/lib/python3.10/site-packages/diffusers/models/downsampling.py:136: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert hidden_states.shape[1] == self.channels
/home/tiennv/.conda/envs/stable_py10/lib/python3.10/site-packages/diffusers/models/downsampling.py:145: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert hidden_states.shape[1] == self.channels


In [5]:
module

ControlNetWrapper(
  original_name=ControlNetWrapper
  (controlnet): ControlNetModel(
    original_name=ControlNetModel
    (conv_in): Conv2d(original_name=Conv2d)
    (time_proj): Timesteps(original_name=Timesteps)
    (time_embedding): TimestepEmbedding(
      original_name=TimestepEmbedding
      (linear_1): Linear(original_name=Linear)
      (act): SiLU(original_name=SiLU)
      (linear_2): Linear(original_name=Linear)
    )
    (add_time_proj): Timesteps(original_name=Timesteps)
    (add_embedding): TimestepEmbedding(
      original_name=TimestepEmbedding
      (linear_1): Linear(original_name=Linear)
      (act): SiLU(original_name=SiLU)
      (linear_2): Linear(original_name=Linear)
    )
    (controlnet_cond_embedding): ControlNetConditioningEmbedding(
      original_name=ControlNetConditioningEmbedding
      (conv_in): Conv2d(original_name=Conv2d)
      (blocks): ModuleList(
        original_name=ModuleList
        (0): Conv2d(original_name=Conv2d)
        (1): Conv2d(original

## cnext

In [2]:
from safetensors.torch import load_file
import torch
from sdserve.models.cnext.wrapper import ControlNeXt, ControlNeXtXL

def load_safetensors(model, safetensors_path, strict=True, load_weight_increasement=False):
    if not load_weight_increasement:
        state_dict = load_file(safetensors_path)
        model.load_state_dict(state_dict, strict=strict)
    else:
        state_dict = load_file(safetensors_path)
        pretrained_state_dict = model.state_dict()
        for k in state_dict.keys():
            state_dict[k] = state_dict[k] + pretrained_state_dict[k]
        model.load_state_dict(state_dict, strict=False)

CONTROLNET_CONFIG  = {
    'in_channels': [128, 128],
    'out_channels': [128, 256],
    'groups': [4, 8],
    'time_embed_dim': 256,
    'final_out_channels': 320,
}

# load from checkpoint
controlnet = ControlNeXtXL(**CONTROLNET_CONFIG)
load_safetensors(controlnet, 'pretrained/ControlAny-SDXL/vidit_depth/controlnet.safetensors', strict=False)
# dummy inputs
image = torch.randn((1, 3, 512, 512), dtype=torch.float32)
timestep = torch.randint(low=0, high=10, size=(1, ), dtype=torch.int32)
dummy_inputs = (image, timestep)

In [3]:
torch.jit.trace(controlnet, dummy_inputs)

/home/tiennv/.conda/envs/stable_py10/lib/python3.10/site-packages/diffusers/models/downsampling.py:135: FutureWarning: `scale` is deprecated and will be removed in version 1.0.0. The `scale` argument is deprecated and will be ignored. Please remove it, as passing it will raise an error in the future. `scale` should directly be passed while calling the underlying pipeline component i.e., via `cross_attention_kwargs`.
  deprecate("scale", "1.0.0", deprecation_message)
/home/tiennv/.conda/envs/stable_py10/lib/python3.10/site-packages/diffusers/models/downsampling.py:136: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert hidden_states.shape[1] == self.channels
/home/tiennv/.conda/envs/stable_py10/lib/python3.10/site-packages/diffusers/models/downsampling.py:145: TracerW

ControlNeXtXL(
  original_name=ControlNeXtXL
  (time_proj): Timesteps(original_name=Timesteps)
  (time_embedding): TimestepEmbedding(
    original_name=TimestepEmbedding
    (linear_1): Linear(original_name=Linear)
    (act): SiLU(original_name=SiLU)
    (linear_2): Linear(original_name=Linear)
  )
  (down_res): ModuleList(
    original_name=ModuleList
    (0): ResnetBlock2D(
      original_name=ResnetBlock2D
      (norm1): GroupNorm(original_name=GroupNorm)
      (conv1): Conv2d(original_name=Conv2d)
      (time_emb_proj): Linear(original_name=Linear)
      (norm2): GroupNorm(original_name=GroupNorm)
      (dropout): Dropout(original_name=Dropout)
      (conv2): Conv2d(original_name=Conv2d)
      (nonlinearity): SiLU(original_name=SiLU)
    )
    (1): ResnetBlock2D(
      original_name=ResnetBlock2D
      (norm1): GroupNorm(original_name=GroupNorm)
      (conv1): Conv2d(original_name=Conv2d)
      (time_emb_proj): Linear(original_name=Linear)
      (norm2): GroupNorm(original_name=Gro

In [4]:
from safetensors.torch import load_file
import torch
from sdserve.models.cnext.wrapper import ControlNeXt

def load_safetensors(model, safetensors_path, strict=True, load_weight_increasement=False):
    if not load_weight_increasement:
        state_dict = load_file(safetensors_path)
        model.load_state_dict(state_dict, strict=strict)
    else:
        state_dict = load_file(safetensors_path)
        pretrained_state_dict = model.state_dict()
        for k in state_dict.keys():
            state_dict[k] = state_dict[k] + pretrained_state_dict[k]
        model.load_state_dict(state_dict, strict=False)


# load from checkpoint
controlnet = ControlNeXt()
load_safetensors(controlnet, 'weights/cnext15-pretrained/deepfashion_caption/controlnet.safetensors')
# dummy inputs
image = torch.randn((1, 3, 512, 512), dtype=torch.float32)
timestep = torch.randint(low=0, high=10, size=(1, ), dtype=torch.int32)
dummy_inputs = (image, timestep)

## sd 1.5


In [13]:
from inspect import signature
 
forward_parameters = signature(controlnet.forward).parameters
forward_inputs_set = list(forward_parameters.keys())
forward_inputs_set

['sample', 'timestep']